In [ ]:
import pickle
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import sklearn.manifold
import numpy as np
import pandas as pd
import re
import multiprocessing
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import collections
import nltk
nltk.download('punkt')

In [ ]:
file = open("weat.txt", "r" ) 
file=file.read()

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(file.lower())

In [ ]:
import pandas as pd 

df1 = pd.read_csv("regiona.csv")

In [ ]:
file_ = list(df1.Text.values)
file2 = ''.join(map(str, file_))

In [ ]:
#Split the corpus into sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences2 = tokenizer.tokenize(file2)

In [ ]:
print("Corpus is now {0} characters long".format(len(file2)))

In [ ]:
def sentence_to_wordlist(raw):
    clean=re.sub('[\n]',' ',raw.lower())
    clean = re.sub("[^a-zA-Z]"," ", raw.lower())
    words = clean.split()
    #words = [ps.stem(w) for w in words if not w in stop_words]  
    words = [w for w in words if not w in stop_words] 
    return words

In [ ]:
sentences = []
for raw_sentence in raw_sentences2:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))
#print(sentences[1])

In [ ]:
token_count = sum([len(sentence) for sentence in sentences])
print("The corpus contains {0:,} tokens".format(token_count))

In [ ]:
def getWordList(sentences,minCount):
    wordList = []
    count = Counter()
    for sentence in sentences:
        for word in sentence:
            count[word] += 1
    for word,c in count.items():
        if c >= minCount:
            wordList.append(word)
    return wordList

In [ ]:
WordList=getWordList(sentences,minCount=5) # for min frequency of a word which should be 5
#print(WordList)

In [ ]:
model = Word2Vec(sentences,vector_size=300,window=5,min_count=5,seed=102,workers=multiprocessing.cpu_count(),batch_words=1)

In [ ]:
#word_frequency
w2c = dict()
for item in model.wv.vocab:
    w2c[item]=model.wv.vocab[item].count
w2cSorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))
# reverse output word and their frequency

In [ ]:
def target_attribute(target_one,target_two, target_one_words, attribute_one,attribute_two, attribute_one_words, target_two_words, attribute_two_words):
    cos=[]
    s=0
    s1=[]
    s2=[]
    S=[]
    n=0
    for i in range(0, len(target_one_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_one_words[i][:-1]
                at1 = attribute_one_words[k][:-1]
                try:
                    cos1= model.wv.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_one_words[i][:-1]
                at2 = attribute_two_words[k][:-1]
                try:
                    cos2= model.wv.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s1.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
            n=n+1
    for i in range(0, len(target_two_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_two_words[i][:-1]
                at1 = attribute_one_words[k][:-1]
                try:
                    cos1= model.wv.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_two_words[i][:-1]
                at2 = attribute_two_words[k][:-1]
                try:
                    cos2= model.wv.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s2.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
    s=np.sum(s1)-np.sum(s2)
    stdev=np.std(S)
    print(target_one + ' vs ' + target_two  + ' , ' +attribute_one + ' vs ' + attribute_two +', d = ' + str(s/(stdev*n)))
    #print(str(s/(stdev*n)))

In [ ]:
for i in range(len(raw_sentences)-30):
    words=raw_sentences[i*4].split()
    target_one = words[0][:-1]
    target_one_words = words[1:]
    words1=raw_sentences[i*4+1].split()
    target_two = words1[0][:-1]
    target_two_words = words1[1:]
    words2=raw_sentences[i*4+2].split()
    attribute_one = words2[0][:-1]
    attribute_one_words = words2[1:]
    words3 = raw_sentences[i*4+3].split()
    attribute_two = words3[0][:-1]
    attribute_two_words = words3[1:]
    
    target_attribute(target_one,target_two, target_one_words, attribute_one,attribute_two, attribute_one_words, target_two_words, attribute_two_words)